In [ ]:
library(tidyverse)

# Introduction 

Some of the advantages of purrr functions over the equivalents in base R:

* The first argument is always the data, so purrr works naturally with the pipe.

* All purrr functions are type-stable. They always return the advertised output type (`map()` returns lists; `map_dbl()` returns double vectors), or they throw an error.

* All `map()` functions either accept function, formulas (used for succinctly generating anonymous functions), a character vector (used to extract components by name), or a numeric vector (used to extract by position)

# Common

The `map(.x, .f)` functions transforms each element of the vector `.x` with the function `.f`, returning a vector defined by the suffix (`_lgl`, `_chr()` etc). `walk()` is a variant for functions called primarily for their side-effects; it returns `.x` invisibly.

As well as functions, `.f`, can take numbers and characters (used as a shorthand for `[[`), and formulas (used as a succint function definition). 

# Arguments

`.x`	
A list or atomic vector.

`.f`	
A function, formula, or vector (not necessarily atomic).

* If a **function**, it is used as is.


* If a **formula**, e.g. ~ .x + 2, it is converted to a function. There are three ways to refer to the arguments:

    * For a single argument function, use `.`

    * For a two argument function, use `.x` and `.y`

    * For more arguments, use `..1`, `..2`, `..3` etc

  This syntax allows you to create very compact anonymous functions.
 
 
* If **character vector**, **numeric vector**, or **list**, it is converted to an extractor function. Character vectors index by name and numeric vectors index by position; use a list to index by position and name at different levels. If a component is not present, the value of .default will be returned.

`...`	
Additional arguments passed on to the mapped function.

# Function

<b style = 'color:red'>NOTE: `purrr` functions preverse name</b>

In [4]:
c(a = 1, b = 2) %>% map_dbl(~ . * 2L)

a b 
2 4

In [6]:
list(a = 1, b = 2) %>% map_dbl(~ . * 2L)

a b 
2 4

In [12]:
players <- c('Pikachu', 'Pokemon', 'Charizard')

decor <- function(value) str_c('xXx-', value, '-xXx')


# Map each value of vector `players` with function `decor`
# The result is a character vector (`map_chr`)
players %>% map_chr(decor)

[1] "xXx-Pikachu-xXx"   "xXx-Pokemon-xXx"   "xXx-Charizard-xXx"

In [26]:
# Passing argument to the map function
decor_complex <- function(value, prefix, suffix) str_c(prefix, value, suffix)

# as argument
players %>% map_chr(decor_complex, 'VNC ', '-xXx')

# as keyword arguments (equivalent)
players %>% map_chr(decor_complex, suffix = '-xXx', prefix = 'VNC ')

[1] "VNC Pikachu-xXx"   "VNC Pokemon-xXx"   "VNC Charizard-xXx"

[1] "VNC Pikachu-xXx"   "VNC Pokemon-xXx"   "VNC Charizard-xXx"

# Formula

For a function of 1 argument, use `.`

In [14]:
players %>% map_chr(~ str_c('xXx-', ., '-xXx'))

[1] "xXx-Pikachu-xXx"   "xXx-Pokemon-xXx"   "xXx-Charizard-xXx"

For a function of 2 arguments, use `.x` and `.y`

In [30]:
clan <- c('VNC', 'Dirilis', 'King Allool')
player <- c('Meomeo888', 'ETOGRUL', 'xXx-Hadi-xXx')

map2(clan, player, ~ str_c(.x, ': ', .y))

[[1]]
[1] "VNC: Meomeo888"

[[2]]
[1] "Dirilis: ETOGRUL"

[[3]]
[1] "King Allool: xXx-Hadi-xXx"

# Extractor funtions: character vector, numeric vector and list

In [29]:
#extract the value of name 'a' for each list. replace missing value with '???'

l1 <- list(list(a = 1L), list(a = NULL, b = 2L), list(b = 3L))

l1 %>% map('a', .default = '???')

[[1]]
[1] 1

[[2]]
[1] "???"

[[3]]
[1] "???"

Using a vector or list to extract value in a deeply nested list

In [41]:
VNC <- list(
    #The fist value is average damage, the second value is the level
    name = 'VN Champions',
    stat = c(100, 99),
    information = c(motto = 'Most powerful', rich = 'very rich')
    
)

Dirilis <- list(
    name = 'Dirilis',
    stat = c(75, 33),
    information = c(motto = 'Play and have fun', rich = 'quite')
)

clan <- list(VNC, Dirilis)

clan

[[1]]
[[1]]$name
[1] "VN Champions"

[[1]]$stat
[1] 100  99

[[1]]$information
          motto            rich 
"Most powerful"     "very rich" 


[[2]]
[[2]]$name
[1] "Dirilis"

[[2]]$stat
[1] 75 33

[[2]]$information
              motto                rich 
"Play and have fun"             "quite"

In [49]:
# get the first value of attribute 'stat' from each list
clan %>% map_dbl(list('stat', 1))  # using list

[1] 100  75

In [50]:
clan %>% map_chr(c('information', 'motto'))  # using character vector
#equivalent
clan %>% map_chr(c(3, 1))                    # using numeric vector

[1] "Most powerful"     "Play and have fun"

[1] "Most powerful"     "Play and have fun"

# Map variants

|                     | List	|Atomic	        |Same type	|Nothing|
|:--------------------|:--------|:--------------|:----------|:------|
|One argument	      | `map()`	|`map_lgl()`, …	|`modify()`	|`walk()` |
|Two arguments	      | `map2()`	|`map2_lgl()`, …	|`modify2()`	|`walk2()`|
|One argument + index | `imap()`	|`imap_lgl()`, …	|`imodify()`	|`iwalk()`|
|N arguments	      | `pmap()`	|`pmap_lgl()`, …	|  —	    |`pwalk()`|

# NOTES

Note there’s a subtle difference between placing extra arguments inside an anonymous function compared with passing them to `map()`. Putting them in an anonymous function means that they will be evaluated every time `f()` is executed, not just once when you call `map()`. This is easiest to see if we make the additional argument random:

In [9]:
f <- function(x, y) y

1:5 %>% map_dbl(f, runif(1))

1:5 %>% map_dbl(~ f(., runif(1)))

[1] 0.7547535 0.7547535 0.7547535 0.7547535 0.7547535

[1] 0.4573411 0.3853449 0.4799755 0.2494051 0.4734021